In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/elo-ratings-1/ELO_Ratings/ManCity.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Liverpool.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/werder.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Milan.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Olympiacos.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Marseille.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Panathinaikos.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Lyon.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Dortmund.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Inter.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Benfica.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Wolfsburg.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Lille.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Celtic.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Napoli.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Monaco.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Basel.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/ParisSG.csv
/kaggle/input/elo-ratings-1/ELO_Ratings/Bayern.csv
/kaggle/inpu

In [19]:
# Load main dataset
df = pd.read_csv('/kaggle/input/train-data/train_data_till_match_date.csv')
df['date'] = pd.to_datetime(df['date'])

In [20]:
df.head()

,season,round,match_no,team1,team2,date,winner,attack_strength_ratio,ucl_attack_ratio,league_attack_ratio,...,ucl_defense_ratio,league_defense_ratio,overall_form_ratio,ucl_form_ratio,league_form_ratio,goal_diff_ratio,league_goal_diff_ratio,ucl_experience_diff,team1_wins,result
0,2004,Round Of 16,Match 1,real madrid,juventus,2005-02-22,juventus,1.418759,2.00000,0.949518,...,0.250000,0.690236,0.849920,0.875000,0.784257,0.000000,-0.412827,0,0.0,team2_win
1,2004,Round Of 16,Match 2,liverpool,bayer leverkusen,2005-02-22,liverpool,0.654478,0.62069,0.705026,...,2.000000,1.112821,0.981320,1.083333,0.775312,-0.214286,-0.491758,-7,1.0,team1_win
2,2004,Round Of 16,Match 3,psv eindhoven,monaco,2005-02-22,psv eindhoven,0.389959,0.70000,0.000000,...,0.714286,0.851852,0.740506,1.083333,0.000000,-0.714286,-0.851852,0,1.0,team1_win
3,2004,Round Of 16,Match 4,bayern munich,arsenal,2005-02-22,bayern munich,1.122047,1.25000,0.976891,...,1.500000,1.062069,1.197735,1.300000,1.037946,0.857143,0.011905,0,1.0,team1_win
4,2004,Round Of 16,Match 5,barcelona,chelsea,2005-02-23,chelsea,1.030227,1.00000,1.069091,...,0.714286,0.490196,0.972549,1.000000,0.924324,-0.285714,-0.220000,0,0.0,team2_win


In [21]:
def update_inter_matches(row):
    if row['team1'] in inter_variations or row['team2'] in inter_variations:
        if row['winner'] == row['team1']:
            row['team1_wins'] = 1.0
            row['result'] = 'team1_win'
        elif row['winner'] == row['team2']:
            row['team1_wins'] = 0.0
            row['result'] = 'team2_win'
        else:
            # Draw or other result
            row['team1_wins'] = 0.5
            row['result'] = 'draw'
    return row

In [22]:
inter_variations = ['inter', 'internazionale']
df.replace('internazionale', 'inter', inplace=True)

In [23]:
# Apply the function
df = df.apply(update_inter_matches, axis=1)

In [24]:
df[:10]

,season,round,match_no,team1,team2,date,winner,attack_strength_ratio,ucl_attack_ratio,league_attack_ratio,...,ucl_defense_ratio,league_defense_ratio,overall_form_ratio,ucl_form_ratio,league_form_ratio,goal_diff_ratio,league_goal_diff_ratio,ucl_experience_diff,team1_wins,result
0,2004,Round Of 16,Match 1,real madrid,juventus,2005-02-22,juventus,1.418759,2.000000,0.949518,...,0.250000,0.690236,0.849920,0.875000,0.784257,0.000000,-0.412827,0,0.0,team2_win
1,2004,Round Of 16,Match 2,liverpool,bayer leverkusen,2005-02-22,liverpool,0.654478,0.620690,0.705026,...,2.000000,1.112821,0.981320,1.083333,0.775312,-0.214286,-0.491758,-7,1.0,team1_win
2,2004,Round Of 16,Match 3,psv eindhoven,monaco,2005-02-22,psv eindhoven,0.389959,0.700000,0.000000,...,0.714286,0.851852,0.740506,1.083333,0.000000,-0.714286,-0.851852,0,1.0,team1_win
3,2004,Round Of 16,Match 4,bayern munich,arsenal,2005-02-22,bayern munich,1.122047,1.250000,0.976891,...,1.500000,1.062069,1.197735,1.300000,1.037946,0.857143,0.011905,0,1.0,team1_win
4,2004,Round Of 16,Match 5,barcelona,chelsea,2005-02-23,chelsea,1.030227,1.000000,1.069091,...,0.714286,0.490196,0.972549,1.000000,0.924324,-0.285714,-0.220000,0,0.0,team2_win
5,2004,Round Of 16,Match 6,manchester united,milan,2005-02-23,milan,1.059614,1.272727,0.778305,...,0.300000,0.870342,0.709257,0.687500,0.760829,-0.571429,-0.496973,0,0.0,team2_win
6,2004,Round Of 16,Match 7,werder bremen,lyon,2005-02-23,lyon,0.826923,0.600000,1.507692,...,0.888889,0.419355,0.835692,0.812500,0.894394,-1.285714,0.032967,0,0.0,team2_win
7,2004,Round Of 16,Match 8,porto,inter,2005-02-23,inter,0.206847,0.333333,0.000000,...,0.571429,1.103448,0.435000,0.600000,0.000000,-1.857143,-0.862069,0,0.0,team2_win
8,2004,Quarter Finals,Match 1,liverpool,juventus,2005-04-05,liverpool,1.258668,1.750000,0.852500,...,0.666667,0.677419,0.915511,1.000000,0.722948,0.444444,-0.560484,0,1.0,team1_win
9,2004,Quarter Finals,Match 2,chelsea,bayern munich,2005-04-05,chelsea,0.948364,1.000000,0.882353,...,1.000000,2.540659,1.201014,1.230769,1.149733,0.000000,0.330252,0,1.0,team1_win


In [2]:
import glob

In [3]:
elo_dict = {}
elo_dir = '/kaggle/input/elo-ratings-1/ELO_Ratings'

for filepath in glob.glob(f"{elo_dir}/*.csv"):
    team = filepath.split('/')[-1].replace('.csv', '').lower()
    elo_df = pd.read_csv(filepath)
    elo_df['From'] = pd.to_datetime(elo_df['From'])
    elo_df['To'] = pd.to_datetime(elo_df['To'])
    elo_dict[team] = elo_df

In [4]:
# Map to ClubElo URL format (adjust as needed, check on clubelo.com)
elo_url_map = {
    'ajax': 'Ajax',
    'apoel': 'APOEL',
    'arsenal': 'Arsenal',
    'atletico madrid': 'Atletico',
    'barcelona': 'Barcelona',
    'basel': 'Basel',
    'bayer leverkusen': 'Leverkusen',
    'bayern munich': 'Bayern',
    'benfica': 'Benfica',
    'bordeaux': 'Bordeaux',
    'borussia dortmund': 'Dortmund',
    'celtic': 'Celtic',
    'chelsea': 'Chelsea',
    'copenhagen': 'Copenhagen',
    'cska moscow': 'CSKAMoskva',
    'dynamo kyiv': 'DynamoKyiv',
    'fenerbahçe': 'Fenerbahce',  # Note: remove accents
    'fiorentina': 'Fiorentina',
    'galatasaray': 'Galatasaray',
    'gent': 'Gent',
    'inter': 'Inter',
    'juventus': 'Juventus',
    'leicester city': 'Leicester',
    'lille': 'Lille',
    'liverpool': 'Liverpool',
    'lyon': 'Lyon',
    'malaga': 'Malaga',
    'manchester city': 'ManCity',
    'manchester united': 'ManUnited',
    'marseille': 'Marseille',
    'milan': 'Milan',
    'monaco': 'Monaco',
    'napoli': 'Napoli',
    'olympiacos': 'Olympiacos',
    'panathinaikos': 'Panathinaikos',
    'paris saint-germain': 'ParisSG',
    'porto': 'Porto',
    'psv eindhoven': 'PSV',
    'rangers': 'Rangers',
    'real madrid': 'RealMadrid',
    'roma': 'Roma',
    'schalke 04': 'Schalke',
    'sevilla': 'Sevilla',
    'shakhtar donetsk': 'Shakhtar',
    'sporting cp': 'sporting',
    'stuttgart': 'Stuttgart',
    'tottenham hotspur': 'Tottenham',
    'valencia': 'Valencia',
    'villarreal': 'Villarreal',
    'werder bremen': 'werder',
    'wolfsburg': 'Wolfsburg',
    'zenit saint petersburg': 'Zenit'
}

In [5]:
def get_elo(team, date):
    if team.lower() not in elo_url_map:
        return None
    elo_df = elo_dict[elo_url_map[team].lower()]
    mask = (elo_df['From'] <= date) & (elo_df['To'] >= date)
    if mask.sum() == 0:
        # If no exact match, take the most recent before date
        mask = elo_df['From'] <= date
        if mask.sum() == 0:
            return None
        return elo_df[mask].iloc[-1]['Elo']
    else:
        return elo_df[mask].iloc[0]['Elo']

In [29]:
# Add Elo ratings for team1 and team2
df['elo_team1'] = df.apply(lambda x: get_elo(x['team1'], x['date']), axis=1)
df['elo_team2'] = df.apply(lambda x: get_elo(x['team2'], x['date']), axis=1)

In [30]:
# Add Elo difference
df['elo_diff'] = df['elo_team1'] - df['elo_team2'] 

In [34]:
sec =set(df['elo_diff'].isnull())
sec

{False, True}

In [9]:
ucl_df = pd.read_csv('/kaggle/input/location/ucl.csv')

In [35]:
import pandas as pd

# Load your datasets
train_df = df.copy()

train_df['date'] = pd.to_datetime(train_df['date'])
ucl_df['Date'] = pd.to_datetime(ucl_df['Date'], dayfirst=True)

# Initialize the new column
train_df['team1_home'] = -1  # -1 means not found

# Function to fuzzy match team names using word-based partial matching
def fuzzy_match(team_name, ucl_team_name):
    # Split both names into words and check if any word in team_name is in ucl_team_name
    team_words = team_name.lower().split()
    ucl_words = ucl_team_name.lower().split()
    return any(word in ucl_words for word in team_words)

# Iterate over each match in train_df
for idx, row in train_df.iterrows():
    date = row['date']
    team1 = row['team1']
    team2 = row['team2']
    season = row['season']
    
    # Find matching rows in ucl_df for the same date and season
    ucl_matches = ucl_df[
        (ucl_df['Date'] == date) &
        (ucl_df['season'] == season)
    ]
    
    for _, ucl_row in ucl_matches.iterrows():
        # Check if team1 matches the home team in ucl_row
        if fuzzy_match(team1, ucl_row['Team']):
            # Check if team2 matches the away team in ucl_row
            if fuzzy_match(team2, ucl_row['Opponent']):
                # If so, set team1_home to 1 if Location is Home, else 0 (but in ucl.csv, Team is always home)
                train_df.at[idx, 'team1_home'] = 1 if ucl_row['Location'] == 'Home' else 0
                break
        # Also check if team2 matches the home team in ucl_row
        elif fuzzy_match(team2, ucl_row['Team']):
            if fuzzy_match(team1, ucl_row['Opponent']):
                train_df.at[idx, 'team1_home'] = 0 if ucl_row['Location'] == 'Home' else 1
                break

# Optionally, handle cases where no match was found (keep as -1, or set to NaN)
train_df['team1_home'] = train_df['team1_home'].replace(-1, pd.NA)



In [36]:
ucl_df['Location'].unique()

array(['Home', 'Away'], dtype=object)

In [40]:
train_df['elo_ratio'] = train_df['elo_team1']/train_df['elo_team2']

In [41]:
train_df.head()

,season,round,match_no,team1,team2,date,winner,attack_strength_ratio,ucl_attack_ratio,league_attack_ratio,...,goal_diff_ratio,league_goal_diff_ratio,ucl_experience_diff,team1_wins,result,elo_team1,elo_team2,elo_diff,team1_home,elo_ratio
0,2004,Round Of 16,Match 1,real madrid,juventus,2005-02-22,juventus,1.418759,2.00000,0.949518,...,0.000000,-0.412827,0,0.0,team2_win,1881.471558,1838.992065,42.479492,1,1.023099
1,2004,Round Of 16,Match 2,liverpool,bayer leverkusen,2005-02-22,liverpool,0.654478,0.62069,0.705026,...,-0.214286,-0.491758,-7,1.0,team1_win,1784.444458,1811.775757,-27.331299,1,0.984915
2,2004,Round Of 16,Match 3,psv eindhoven,monaco,2005-02-22,psv eindhoven,0.389959,0.70000,0.000000,...,-0.714286,-0.851852,0,1.0,team1_win,1806.328735,1841.876465,-35.547729,1,0.980700
3,2004,Round Of 16,Match 4,bayern munich,arsenal,2005-02-22,bayern munich,1.122047,1.25000,0.976891,...,0.857143,0.011905,0,1.0,team1_win,1839.320801,1902.455933,-63.135132,1,0.966814
4,2004,Round Of 16,Match 5,barcelona,chelsea,2005-02-23,chelsea,1.030227,1.00000,1.069091,...,-0.285714,-0.220000,0,0.0,team2_win,1907.777344,1942.089966,-34.312622,1,0.982332


In [38]:
path = '/kaggle/working/final_train.csv'
train_df.to_csv(path)

In [7]:
df_test = pd.read_csv('/kaggle/input/test-round-16/test_data_round_16.csv')

In [8]:
df_test.head()

,season,round,match_no,team1,team2,date,attack_strength_ratio,ucl_attack_ratio,league_attack_ratio,defense_strength_ratio,ucl_defense_ratio,league_defense_ratio,overall_form_ratio,ucl_form_ratio,league_form_ratio,goal_diff_ratio,league_goal_diff_ratio,ucl_experience_diff
0,2017,Round Of 16 Matchups,Match 1,juventus,tottenham hotspur,2018-02-13,0.738025,0.466667,1.264368,1.015385,0.800000,1.500000,0.843510,0.687500,1.337100,-1.500000,0.788889,0.0
1,2017,Round Of 16 Matchups,Match 2,basel,manchester city,2018-02-13,0.482602,0.846154,0.000000,0.750958,0.800000,0.689655,0.565854,0.800000,0.000000,-0.500000,-1.758621,-6.0
2,2017,Round Of 16 Matchups,Match 3,porto,liverpool,2018-02-14,0.469256,0.652174,0.000000,0.773399,0.600000,1.206897,0.586366,0.833333,0.000000,-2.000000,-1.034483,0.0
3,2017,Round Of 16 Matchups,Match 4,sevilla,manchester united,2018-02-21,0.778379,0.866667,0.651042,0.380779,0.307692,0.560684,0.650853,0.590909,0.799825,-1.692308,-1.372917,-7.0
4,2017,Round Of 16 Matchups,Match 5,real madrid,paris saint-germain,2018-02-14,0.649469,0.680000,0.589319,0.598356,0.571429,0.634483,0.794777,0.866667,0.630435,-1.833333,-1.779610,0.0


In [9]:
df_test.replace('internazionale', 'inter', inplace=True)

In [17]:
# Add Elo ratings for team1 and team2
df_test['elo_team1'] = df_test.apply(lambda x: get_elo(x['team1'], x['date']), axis=1)
df_test['elo_team2'] = df_test.apply(lambda x: get_elo(x['team2'], x['date']), axis=1)

In [18]:
df_test.head()

,season,round,match_no,team1,team2,date,attack_strength_ratio,ucl_attack_ratio,league_attack_ratio,defense_strength_ratio,ucl_defense_ratio,league_defense_ratio,overall_form_ratio,ucl_form_ratio,league_form_ratio,goal_diff_ratio,league_goal_diff_ratio,ucl_experience_diff,elo_team1,elo_team2
0,2017,Round Of 16 Matchups,Match 1,juventus,tottenham hotspur,2018-02-13,0.738025,0.466667,1.264368,1.015385,0.800000,1.500000,0.843510,0.687500,1.337100,-1.500000,0.788889,0.0,1968.273560,1926.842407
1,2017,Round Of 16 Matchups,Match 2,basel,manchester city,2018-02-13,0.482602,0.846154,0.000000,0.750958,0.800000,0.689655,0.565854,0.800000,0.000000,-0.500000,-1.758621,-6.0,1706.819946,1975.623535
2,2017,Round Of 16 Matchups,Match 3,porto,liverpool,2018-02-14,0.469256,0.652174,0.000000,0.773399,0.600000,1.206897,0.586366,0.833333,0.000000,-2.000000,-1.034483,0.0,1779.842896,1897.914062
3,2017,Round Of 16 Matchups,Match 4,sevilla,manchester united,2018-02-21,0.778379,0.866667,0.651042,0.380779,0.307692,0.560684,0.650853,0.590909,0.799825,-1.692308,-1.372917,-7.0,1755.272339,1899.502197
4,2017,Round Of 16 Matchups,Match 5,real madrid,paris saint-germain,2018-02-14,0.649469,0.680000,0.589319,0.598356,0.571429,0.634483,0.794777,0.866667,0.630435,-1.833333,-1.779610,0.0,1992.745972,1906.519653


In [21]:
train_df = df_test.copy()
train_df['date'] = pd.to_datetime(train_df['date'])
ucl_df['Date'] = pd.to_datetime(ucl_df['Date'], dayfirst=True)

# Initialize the new column
train_df['team1_home'] = -1  # -1 means not found

# Function to fuzzy match team names using word-based partial matching
def fuzzy_match(team_name, ucl_team_name):
    # Split both names into words and check if any word in team_name is in ucl_team_name
    team_words = team_name.lower().split()
    ucl_words = ucl_team_name.lower().split()
    return any(word in ucl_words for word in team_words)

# Iterate over each match in train_df
for idx, row in train_df.iterrows():
    date = row['date']
    team1 = row['team1']
    team2 = row['team2']
    season = row['season']
    
    # Find matching rows in ucl_df for the same date and season
    ucl_matches = ucl_df[
        (ucl_df['Date'] == date) &
        (ucl_df['season'] == season)
    ]
    
    for _, ucl_row in ucl_matches.iterrows():
        # Check if team1 matches the home team in ucl_row
        if fuzzy_match(team1, ucl_row['Team']):
            # Check if team2 matches the away team in ucl_row
            if fuzzy_match(team2, ucl_row['Opponent']):
                # If so, set team1_home to 1 if Location is Home, else 0 (but in ucl.csv, Team is always home)
                train_df.at[idx, 'team1_home'] = 1 if ucl_row['Location'] == 'Home' else 0
                break
        # Also check if team2 matches the home team in ucl_row
        elif fuzzy_match(team2, ucl_row['Team']):
            if fuzzy_match(team1, ucl_row['Opponent']):
                train_df.at[idx, 'team1_home'] = 0 if ucl_row['Location'] == 'Home' else 1
                break

# Optionally, handle cases where no match was found (keep as -1, or set to NaN)
train_df['team1_home'] = train_df['team1_home'].replace(-1, pd.NA)



In [22]:
test_df=train_df.copy()
test_df.head()

,season,round,match_no,team1,team2,date,attack_strength_ratio,ucl_attack_ratio,league_attack_ratio,defense_strength_ratio,...,league_defense_ratio,overall_form_ratio,ucl_form_ratio,league_form_ratio,goal_diff_ratio,league_goal_diff_ratio,ucl_experience_diff,elo_team1,elo_team2,team1_home
0,2017,Round Of 16 Matchups,Match 1,juventus,tottenham hotspur,2018-02-13,0.738025,0.466667,1.264368,1.015385,...,1.500000,0.843510,0.687500,1.337100,-1.500000,0.788889,0.0,1968.273560,1926.842407,1
1,2017,Round Of 16 Matchups,Match 2,basel,manchester city,2018-02-13,0.482602,0.846154,0.000000,0.750958,...,0.689655,0.565854,0.800000,0.000000,-0.500000,-1.758621,-6.0,1706.819946,1975.623535,1
2,2017,Round Of 16 Matchups,Match 3,porto,liverpool,2018-02-14,0.469256,0.652174,0.000000,0.773399,...,1.206897,0.586366,0.833333,0.000000,-2.000000,-1.034483,0.0,1779.842896,1897.914062,1
3,2017,Round Of 16 Matchups,Match 4,sevilla,manchester united,2018-02-21,0.778379,0.866667,0.651042,0.380779,...,0.560684,0.650853,0.590909,0.799825,-1.692308,-1.372917,-7.0,1755.272339,1899.502197,1
4,2017,Round Of 16 Matchups,Match 5,real madrid,paris saint-germain,2018-02-14,0.649469,0.680000,0.589319,0.598356,...,0.634483,0.794777,0.866667,0.630435,-1.833333,-1.779610,0.0,1992.745972,1906.519653,1


In [23]:
path = '/kaggle/working/final_test.csv'
test_df.to_csv(path)

In [6]:
df_test_quarter  = pd.read_csv('/kaggle/input/testing-quarter/test_quarter.csv')

In [7]:
df_test_quarter.replace('internazionale', 'inter', inplace=True)
# Add Elo ratings for team1 and team2
df_test_quarter['elo_team1'] = df_test_quarter.apply(lambda x: get_elo(x['team1'], x['date']), axis=1)
df_test_quarter['elo_team2'] = df_test_quarter.apply(lambda x: get_elo(x['team2'], x['date']), axis=1)

In [8]:
df_test_quarter.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Unnamed: 0,season,round,match_no,team1,team2,date,attack_strength_ratio,ucl_attack_ratio,league_attack_ratio,...,ucl_defense_ratio,league_defense_ratio,overall_form_ratio,ucl_form_ratio,league_form_ratio,goal_diff_ratio,league_goal_diff_ratio,ucl_experience_diff,elo_team1,elo_team2
0,8,2017,Quarter Finals Matchups,Match 1,tottenham hotspur,paris saint-germain,2018-04-03,0.645585,0.500000,1.0,...,1.125000,1.0,0.831663,0.789474,1.0,-1.250,0.0,0.0,1936.533936,1914.407227
1,9,2017,Quarter Finals Matchups,Match 2,liverpool,manchester city,2018-04-04,1.090756,1.153846,1.0,...,0.666667,1.0,0.797590,0.733333,1.0,-0.375,0.0,0.0,1910.774414,1995.755859
2,10,2017,Quarter Finals Matchups,Match 3,manchester united,besiktas,2018-04-03,0.766954,0.666667,1.0,...,0.538462,1.0,0.689464,0.619048,1.0,-1.625,0.0,0.0,1892.021973,NaN
3,11,2017,Quarter Finals Matchups,Match 4,shakhtar donetsk,barcelona,2018-04-04,0.769559,0.647059,1.0,...,1.090909,1.0,1.178977,1.250000,1.0,-0.625,0.0,0.0,1764.830688,2052.485107
4,23,2018,Quarter Finals Matchups,Match 1,porto,real madrid,2019-04-09,1.000000,1.000000,1.0,...,1.000000,1.0,1.000000,1.000000,1.0,0.000,0.0,0.0,1826.489868,1890.958496


In [10]:
train_df = df_test_quarter.copy()
train_df['date'] = pd.to_datetime(train_df['date'])
ucl_df['Date'] = pd.to_datetime(ucl_df['Date'], dayfirst=True)

# Initialize the new column
train_df['team1_home'] = -1  # -1 means not found

# Function to fuzzy match team names using word-based partial matching
def fuzzy_match(team_name, ucl_team_name):
    # Split both names into words and check if any word in team_name is in ucl_team_name
    team_words = team_name.lower().split()
    ucl_words = ucl_team_name.lower().split()
    return any(word in ucl_words for word in team_words)

# Iterate over each match in train_df
for idx, row in train_df.iterrows():
    date = row['date']
    team1 = row['team1']
    team2 = row['team2']
    season = row['season']
    
    # Find matching rows in ucl_df for the same date and season
    ucl_matches = ucl_df[
        (ucl_df['Date'] == date) &
        (ucl_df['season'] == season)
    ]
    
    for _, ucl_row in ucl_matches.iterrows():
        # Check if team1 matches the home team in ucl_row
        if fuzzy_match(team1, ucl_row['Team']):
            # Check if team2 matches the away team in ucl_row
            if fuzzy_match(team2, ucl_row['Opponent']):
                # If so, set team1_home to 1 if Location is Home, else 0 (but in ucl.csv, Team is always home)
                train_df.at[idx, 'team1_home'] = 1 if ucl_row['Location'] == 'Home' else 0
                break
        # Also check if team2 matches the home team in ucl_row
        elif fuzzy_match(team2, ucl_row['Team']):
            if fuzzy_match(team1, ucl_row['Opponent']):
                train_df.at[idx, 'team1_home'] = 0 if ucl_row['Location'] == 'Home' else 1
                break

# Optionally, handle cases where no match was found (keep as -1, or set to NaN)
train_df['team1_home'] = train_df['team1_home'].replace(-1, pd.NA)



In [11]:
df_test_quarter = train_df.copy()
df_test_quarter.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Unnamed: 0,season,round,match_no,team1,team2,date,attack_strength_ratio,ucl_attack_ratio,league_attack_ratio,...,league_defense_ratio,overall_form_ratio,ucl_form_ratio,league_form_ratio,goal_diff_ratio,league_goal_diff_ratio,ucl_experience_diff,elo_team1,elo_team2,team1_home
0,8,2017,Quarter Finals Matchups,Match 1,tottenham hotspur,paris saint-germain,2018-04-03,0.645585,0.500000,1.0,...,1.0,0.831663,0.789474,1.0,-1.250,0.0,0.0,1936.533936,1914.407227,<NA>
1,9,2017,Quarter Finals Matchups,Match 2,liverpool,manchester city,2018-04-04,1.090756,1.153846,1.0,...,1.0,0.797590,0.733333,1.0,-0.375,0.0,0.0,1910.774414,1995.755859,1
2,10,2017,Quarter Finals Matchups,Match 3,manchester united,besiktas,2018-04-03,0.766954,0.666667,1.0,...,1.0,0.689464,0.619048,1.0,-1.625,0.0,0.0,1892.021973,NaN,<NA>
3,11,2017,Quarter Finals Matchups,Match 4,shakhtar donetsk,barcelona,2018-04-04,0.769559,0.647059,1.0,...,1.0,1.178977,1.250000,1.0,-0.625,0.0,0.0,1764.830688,2052.485107,<NA>
4,23,2018,Quarter Finals Matchups,Match 1,porto,real madrid,2019-04-09,1.000000,1.000000,1.0,...,1.0,1.000000,1.000000,1.0,0.000,0.0,0.0,1826.489868,1890.958496,<NA>


In [12]:
df_test_quarter.to_csv('final_test_quarter.csv')